In [21]:

import os
from pathlib import Path

import numpy as np
import pandas as pd

from tqdm import tqdm
from models.bert.model_base import BertBase

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
reddit_path = str(Path(os.path.abspath('')) / 'data/reddit_data/')
sentiment_path = str(Path(os.path.abspath('')) / 'data/sentiment/')
ea = pd.read_csv(str(Path(reddit_path) / 'EA.csv'))

In [8]:
bert = BertBase()
ea_sample = ea.head(1000)


In [17]:
ea_sample['sentiment'] = ea_sample['body'].apply(lambda x: bert.tokenize_and_predict(x))

/tmp/ipykernel_978398/684506221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ea_sample['sentiment'] = ea_sample['body'].apply(lambda x: bert.tokenize_and_predict(x))


In [28]:
def batch_sentiment(df, n_splits=100): 
    n_splits = n_splits
    n_samples = len(df)
    fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=int)
    fold_sizes[: n_samples % n_splits] += 1
    current = 0
    for fold_size in tqdm(fold_sizes): 
        start, stop = current, current + fold_size
        slice_df = df.loc[start: stop, :]
        slice_df['sentiment'] = slice_df['body'].apply(lambda x: bert.tokenize_and_predict(x))
        slice_df.to_csv(str(Path(sentiment_path) / 'ea.csv'), mode='a')
        current = stop